In [1]:
%load_ext sql
%sql sqlite:///store.db

![scheme](scheme.png)

## Задание 1
Вывести страну, где популярнее всего группа Iron Maiden (т.е. треков куплено больше, чем в других странах). Также вывести кол-во купленных треков.

In [2]:
%%sql

WITH q AS (
    SELECT * FROM customers c
    JOIN invoices i ON c.customerid = i.customerid
    JOIN invoice_items ii ON i.invoiceid = ii.invoiceid
    JOIN tracks t ON ii.trackid = t.trackid
    JOIN albums al ON t.albumid = al.albumid
    JOIN artists ar ON al.artistid = ar.artistid
    WHERE ar.name = "Iron Maiden"
)
SELECT country, COUNT(*) orders FROM q
WHERE country IS NOT NULL GROUP BY country
ORDER BY orders DESC LIMIT 1;

 * sqlite:///store.db
Done.


Country,orders
USA,34


## Задание 2
Вывести "средний по длине" трек, среди приобретённых компаниями (только клиентами из конкретных компаний) после 22 апреля 2023 года.
"Средний по длине" = максимально близкий к среднему значению множества треков по длине (не медиана!). Если таких треков несколько, вывести первый по алфавиту.

In [3]:
%%sql

WITH q AS (
    SELECT DISTINCT t.* FROM customers c
    JOIN invoices i ON c.customerid = i.customerid
    JOIN invoice_items ii ON i.invoiceid = ii.invoiceid
    JOIN tracks t ON ii.trackid = t.trackid
    WHERE c.company IS NOT NULL AND i.invoicedate > "2023-04-22 00:00:00"
)
SELECT q.*, ABS(milliseconds - (SELECT AVG(milliseconds) FROM q)) deviation
FROM q ORDER BY deviation LIMIT 1;

 * sqlite:///store.db
Done.


TrackId,Name,AlbumId,MediaTypeId,GenreId,Milliseconds,Bytes,UnitPrice,deviation
1601,No Quarter,129,5,1,420493,13656517,0.99,309.3017676767777


## Задание 3

Вывести треки, встречающиеся и в нескольких плейлистах, и в нескольких заказах одновременно.

In [7]:
%%sql

WITH multiplaylist_tracks AS (
    SELECT trackid FROM playlist_track
    GROUP BY trackid HAVING COUNT(playlistid) > 1
),
multiorder_tracks AS (
    SELECT trackid FROM invoice_items
    GROUP BY trackid HAVING COUNT(invoiceid) > 1
)
SELECT t.*
FROM tracks t
JOIN multiplaylist_tracks mp ON t.trackid = mp.trackid
JOIN multiorder_tracks mo ON t.trackid = mo.trackid;

 * sqlite:///store.db
Done.


TrackId,Name,AlbumId,MediaTypeId,GenreId,Milliseconds,Bytes,UnitPrice
894,Sunshine Of Your Love,72,1,6,252891,8225889,2.99
2832,The Woman King,227,3,18,2626376,552893447,1.99
2850,The Fix,228,3,21,2600266,507026323,1.99
2868,Walkabout,230,3,19,2587370,207748198,1.99
3177,Hot Girl,249,3,19,1325458,267836576,1.99
3200,Gay Witch Hunt,251,3,19,1326534,276942637,1.99
3214,Phyllis's Wedding,251,3,22,1271521,258561054,1.99
3223,How to Stop an Exploding Man,228,3,21,2687103,487881159,1.99
3250,Pilot,254,3,19,2484567,492670102,1.99


## Задание 4
Вывести альбом, который является максимальным по продолжительности среди альбомов со стоимостью, большей средней цены за альбом *(цена альбома = сумма цен треков, продолжительность считать аналогично)*.

In [5]:
%%sql

WITH q AS (
    SELECT a.*, t.unitprice, t.milliseconds FROM albums a
    LEFT JOIN tracks t ON a.albumid = t.albumid
),
total_album_prices AS (
    SELECT q.albumid, q.title, q.artistid,
    SUM(unitprice) total_price, SUM(milliseconds)
    total_duration FROM q GROUP BY albumid
)
SELECT * FROM total_album_prices
WHERE total_price > (SELECT AVG(total_price) FROM total_album_prices)
ORDER BY total_duration DESC LIMIT 1;

 * sqlite:///store.db
Done.


albumid,title,artistid,total_price,total_duration
229,"Lost, Season 3",149,50.74,70665582


## Задание 5
Для каждого исполнителя, у которого есть песни нескольких жанров, найти жанр, по которому он заработал наибольшее количество денег, а также вывести процентное соотношение суммы, заработанной исполнителем по этому жанру к общей сумме заработанных денег. Формат вывода: имя исполнителя, жанр, соотношение.

In [15]:
%%sql

WITH artist_earnings_per_genre AS (
  SELECT ar.name AS artistname, g.name AS genrename, SUM(t.unitprice * ii.quantity) as earnings
  FROM genres g
  JOIN tracks t ON t.genreid = g.genreid
  JOIN albums a ON a.albumid = t.albumId
  JOIN artists ar ON ar.artistid = a.artistid
  JOIN invoice_items ii ON t.trackid = ii.trackid
  GROUP BY ar.name, g.name
),

total_earnings AS (
  SELECT artistname, SUM(earnings) As total
  FROM artist_earnings_per_genre
  GROUP BY artistname
),

max_earnings_per_genre AS (
  SELECT artistname, genrename, earnings as max_earnings
  FROM artist_earnings_per_genre efg
  GROUP BY artistname
  HAVING MAX(earnings)
)

SELECT mepg.ArtistName, mepg.GenreName, (mepg.max_earnings)/(te.total) * 100 as Ratio
FROM genres g
JOIN tracks t ON t.genreid = g.genreid
JOIN albums a ON a.albumid = t.albumId
JOIN artists ar ON ar.artistid = a.artistid
JOIN max_earnings_per_genre mepg ON ar.name = mepg.ArtistName
JOIN total_earnings te ON mepg.ArtistName = te.ArtistName
GROUP BY ar.name
HAVING COUNT(DISTINCT g.name) > 1;

 * sqlite:///store.db
Done.


artistname,genrename,Ratio
Amy Winehouse,R&B/Soul,55.55555555555556
Antônio Carlos Jobim,Latin,62.93745346239762
Audioslave,Alternative & Punk,40.78358208955224
Battlestar Galactica,TV Shows,47.368421052631575
Eric Clapton,Blues,71.75558759913481
Faith No More,Alternative & Punk,87.00422790416751
Foo Fighters,Rock,83.36127409891031
Gilberto Gil,Latin,62.490547012856055
Guns N' Roses,Rock,65.52003706277507
Heroes,Drama,75.0
